In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
from tqdm import tqdm

In [2]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(3, 128, 3)
        self.conv2 = nn.Conv2d(128, 128, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(128, 128, 3)
        self.conv4 = nn.Conv2d(128, 128, 3)

        self.fc1 = nn.Linear(128 * 5 * 5, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.elu(self.conv2(F.elu(self.conv1(x)))))
        x = self.pool(F.elu(self.conv4(F.elu(self.conv3(x)))))
        x = x.view(-1, 128 * 5 * 5)
        x = F.elu(self.fc1(x))
        x = self.fc2(x)
        return x


In [4]:
model = Network().to('cuda')

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

def get_accuracy(loader, model):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in loader:
            images, labels = data
            outputs = model(images.to('cuda')).cpu()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

In [5]:
len(trainloader)

782

In [6]:
for epoch in range(50):
    running_loss = 0.0
    # j = 0
    for i, data in tqdm(enumerate(trainloader, 0), total=len(trainloader)):
        # j += 1
        # print(j)
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs.to('cuda'))

        loss = loss_fn(outputs, labels.to('cuda'))
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f'Epoch {epoch + 1}, loss: {running_loss / len(trainloader)}')

    train_acc = get_accuracy(trainloader, model)
    test_acc = get_accuracy(testloader, model)
    print(f'Training Accuracy: {train_acc}, Validation Accuracy: {test_acc}')


100%|██████████| 782/782 [00:28<00:00, 27.38it/s]


Epoch 1, loss: 1.2287844404235215
Training Accuracy: 70.768, Validation Accuracy: 67.12


100%|██████████| 782/782 [00:24<00:00, 32.31it/s]


Epoch 2, loss: 0.8114016235941817
Training Accuracy: 78.942, Validation Accuracy: 72.54


100%|██████████| 782/782 [00:24<00:00, 32.19it/s]


...

Epoch 29, loss: 0.20357139764926951
Training Accuracy: 97.256, Validation Accuracy: 75.62


100%|██████████| 782/782 [00:23<00:00, 32.94it/s]


Epoch 30, loss: 0.18640457295704846
Training Accuracy: 97.23, Validation Accuracy: 75.4


 13%|█▎        | 102/782 [00:03<00:22, 30.67it/s]


KeyboardInterrupt: 